In [1]:
from toy_model import *
from metrics import *
import wandb
import torch
import numpy as np

/opt/anaconda3/envs/toytrans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

wandb.login()

wandb: Currently logged in as: ce24b119 (jerrycloud3316-ai-club-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# to find kl loss between model and these processes
T0_proc1 = np.array([
    [0, 1, 0],
    [0, 0, 1], 
    [0, 0, 0.5]
])
T1_proc1 = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0.5, 0, 0]
])

# Different process
T0_proc2 = np.array([
    [0,1,0],
    [0,0,0],
    [0.5,0,0]
])
T1_proc2 = np.array([
    [0,0,0],
    [0,0,1],
    [0.5,0,0]
])
T0_proc3 = np.array([
    [0,1,0,0],
    [0,0,0,0.5],
    [0.5,0,0,0],
    [0,0,0,0.5]])
T1_proc3 = np.array([
    [0,0,0,0],
    [0,0,0.5,0],
    [0.5,0,0,0],
    [0.5,0,0,0]])   
process1 = MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
process2 = MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2], seed=43)
process3 = MarkovData(n_gen=100, gen_len=32, n_states=4, d_vocab=2, T_list=[T0_proc3, T1_proc3], seed=43)

In [11]:
dataset1=MarkovData(n_gen=5000, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1])
dataset2=MarkovData(n_gen=5000, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2])
merged_dataset = MergeMarkovDatasets(dataset1=dataset1, dataset2=dataset2,mixing_style='random')
for i in range (10):
    print(merged_dataset[i])

{'tokens': tensor([0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 1])}
{'tokens': tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0])}
{'tokens': tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0])}
{'tokens': tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0])}
{'tokens': tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0])}
{'tokens': tensor([1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 1, 1, 0])}
{'tokens': tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1])}
{'tokens': tensor([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 0])}


In [19]:
x=torch.stack(merged_dataset.data)
onegramfreq, onegramcounts, onegramtotal=get_ngram_stats(x, n=1)
print(onegramfreq, onegramcounts, onegramtotal)
twogramfreq, twogramcounts, twogramtotal=get_ngram_stats(x, n=2)
print(twogramfreq, twogramcounts, twogramtotal)
threegramfreq, threegramcounts, threegramtotal=get_ngram_stats(x, n=3)
print(threegramfreq, threegramcounts, threegramtotal)


{'0': 0.625671875, '1': 0.374328125} {'0': 200215, '1': 119785} 320000
{'00': 0.3337322580645161, '01': 0.2922451612903226, '10': 0.29033870967741937, '11': 0.08368387096774194} {'00': 103457, '01': 90596, '10': 90005, '11': 25942} 310000
{'000': 0.12573, '001': 0.20832333333333333, '010': 0.20825666666666667, '011': 0.08375, '100': 0.20673333333333332, '101': 0.08354666666666667, '110': 0.08366, '111': 0.0} {'000': 37719, '001': 62497, '010': 62477, '011': 25125, '100': 62020, '101': 25064, '110': 25098, '111': 0} 300000


In [ ]:
dataset1=MarkovData(n_gen=50, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
dataset2=MarkovData(n_gen=50, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2], seed=43)
test_dataset = MergeMarkovDatasets(dataset1=dataset1, dataset2=dataset2,mixing_style='random')

In [ ]:

metrics_config = MetricsConfig(
    track_markov_kl=True,
    markov_processes=[process1,process2,process3], 
    
    
    track_ngrams=True,
    ngram_data=test_dataset,
    ngram_orders=[1, 2, 3],
    track_previous_token=True,
    track_in_context=False, 
    icl_data=test_dataset,
    icl_k1=5,
    icl_k2=32,
    track_composition=True,
    track_prefix_matching=False)

In [ ]:

model = train_model(
    dataset=merged_dataset,
    n_layers=1,
    d_model=16,
    n_heads=2, 
    attn_only=False,
    act_fn='silu',

    n_epochs=300,
    batch_size=64,
    lr=0.1,

    wandb=True,
    wandb_project_name="ICL",
    save_dir="proc1/seq_len32/X8",
    save_every=20,
    print_every=10,


    metrics_config=metrics_config,
    metrics_log_interval=20
    )
wandb.finish()

Moving model to device:  cpu


  0%|          | 0/300 [00:00<?, ?it/s]

Metrics logged at step 50


  0%|          | 1/300 [00:01<05:56,  1.19s/it]

Metrics logged at step 100
Epoch 1 Validation Loss 0.6199098825454712
Metrics logged at step 150
Metrics logged at step 200


...

100%|█████████▉| 299/300 [06:19<00:01,  1.22s/it]

Metrics logged at step 37350
Epoch 299 Validation Loss 0.3231199085712433
Metrics logged at step 37400
Metrics logged at step 37450


100%|██████████| 300/300 [06:20<00:00,  1.27s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Metrics logged at step 37500
Epoch 300 Samples 8000 Step 124 Training Loss 0.3270306885242462
Epoch 300 Validation Loss 0.3229559063911438


1gram_kl_model_to_true,▁▁▁▁▄▆▆▆▇▇▇▇▇▇▇█▇███████████████████████
1gram_kl_true_to_model,▁▁▁▂▃▃▃▃▄▄▄▄▄▄▄▅▆▆▅▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇██
2gram_kl_model_to_true,▁▁▁▃▃▆▆▆▆▇▇▇▇▇███▇██████▇███████████▇███
2gram_kl_true_to_model,▁▁▁▁▁▁▁▁▂▃▃▄▄▅▅▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇██████
3gram_kl_model_to_true,▁▃▇▇▇█▇▇▇███████████████████████████████
3gram_kl_true_to_model,▁▁▁▁▂▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇████
avg_prev_token_matching,▂▁▁▁▁▁▁▁▂▂▂▃▄▆▇▇▇▇▇▇█▇▇█████████████████
epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██
markov0_to_model_kl,█▇▇▆▅▄▄▄▃▃▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
markov1_to_model_kl,███▆▂▂▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+10,...
